## Attention

Long term dependency를 해결하기 위해 사용함 <br>

### Why 등장?
* Attention mechanism = 주의, 집중
어느 한 문장의 words 중에서도 task 성능에 영향을 끼치는 정도(가중치)가 word마다 다르다.
이러한 단어들을 활용해서 학습을 시키면 좋은 성능의 학습 모델을 만들 수 있지 않을까?
<br>

rnn은 문장의 순차적 특성을 유지.
하지만, 두 정보 사이의 거리가 멀 때 해당 정보를 이용하지 못하는 문제가 발생한다. (<span style="color:red;">Long-term dependency problem</span>)
<br>

rnn은 학습 시 t번째 hidden state를 얻기 위해 t-1번째 hidden state가 필요하기 때문, 즉 순서대로 계산되어야 하기 때문에 <br>
→ <b>병렬 처리 불가능 & 속도가 느림 (Parallelization)</b><br>
→ Tensorflow 같이 gpu 사용에 대해서 매우 치명적인 단점!
<br><br>

### idea
| Attention : 모델이 중요한 부분에 <b>집중</b>하게 만들자!

* decoder에서 출력 단어를 예측하는 매 시점마다 <span style="color:red;">Encoder에서의 전체 입력 문장을 다시 참고</span>
* 동일한 비율로 참고하는 것이 아닌 해당 시점에서 <span style="color:red;">예측해야 할 단어와 연관있는 입력 단어 부분에 좀 더 집중(Attention)해서 보겠다.</span>
<br><br>

### Attention Function
| Attention(Q, K, V) = Attention Value

1. Query에 대해 모든 Key와의 유사도를 각각 구함
2. 이 유사도를 키와 매핑되어 있는 각각의 Value에 반영
3. 유사도가 반영된 Value를 모두 더해서 리턴 (이 값이 Attention Value)

* Query = decoder의 t-1 셀에서의 은닉 상태
* Keys = 모든 Encoder 셀의 은닉 상태들
* Values = 모든 Encoder 셀의 은닉 상태들


In [ ]:
# Attention의 유사도를 구함
w_omega = tf.Variable(tf.random_normal([hidden_size, attention_size], stddev=0.1))
b_omega = tf.Variable(tf.random_normal([attention_size], stddev=0.1))
u_omega = tf.Variable(tf.random_normal([attention_size], stddev=0.1))

with tf.name_scope('v'):
    v = tf.tanh(tf.tensordot(inputs, w_omega, axes=1) + b_omega)

vu = tf.tensordot(v, u_omega, axes=1, name='vu')

# softmax 함수
alphas = tf.nn.softmax(vu, name='alphas')

# Weighted sum
output = tf.reduce_sum(inputs*tf.expand_dims(alphas, -1), 1)


In [1]:
import tensorflow as tf
import numpy as np

In [2]:
# S : 디코딩 입력의 시작을 나타내는 심볼
# E : 디코딩 출력의 끝을 나타내는 심볼
# P : padding token
char_arr = [c for c in 'SEPabcdefghijklmnopqrstuvwxyz단어나무놀이소녀키스사랑']
num_dic = {n:i for i,n in enumerate(char_arr)}
dic_len = len(num_dic)

# 영어를 한글로 번역하기 위한 학습 데이터
seq_data = [['word', '단어'], ['wood','나무'], ['game', '놀이'], ['girl', '소녀'], ['kiss', '키스'], ['love', '사랑']]

def make_batch(seq_data):
    input_batch = []
    output_batch = []
    target_batch = []

    for seq in seq_data:
        # 인코더 셀의 입력값. 입력단어의 글자들을 한글자씩 떼어 배열로 만듦
        # e.g.) w, o, r, d
        input = [num_dic[n] for n in seq[0]]

        # 디코더 셀의 입력값. 시작을 나타내는 S 심볼을 맨 앞에 붙여줌
        # e.g.) S, 단, 어
        output = [num_dic[n] for n in ('S' + seq[1])]

        # 학습을 위해 비교할 디코더 셀의 출력값. 끝나는 것을 알려주기 위해 마지막에 E를 붙인다
        # e.g.) 단, 어, E
        target = [num_dic[n] for n in (seq[1] + 'E')]

        input_batch.append(np.eye(dic_len)[input])
        output_batch.append(np.eye(dic_len)[output])
    
        # 출력값만 one-hot 인코딩이 아님 (Sparse_softmax_cross_entropy_with_logits 사용)
        target_batch.append(target)

    return input_batch, output_batch, target_batch

In [3]:
# 옵션 설정
learning_rate = 0.01
n_hidden = 128
total_epoch = 100

# 입력과 출력의 형태가 one-hot encoding과 같으므로 크기도 같다
n_class = n_input = dic_len

## 신경망 모델 구성
# Seq2Seq 모델은 인코더의 입력과 디코더의 입력의 형식이 같다
# [batch_size, time_steps, input_size]
enc_input = tf.placeholder(tf.float32, [None, None, n_input])
dec_input = tf.placeholder(tf.float32, [None, None, n_input])

# [batch_size, time_steps]
targets = tf.placeholder(tf.int64, [None, None])

## 인코더 셀을 구성한다
with tf.variable_scope('encode'):
    enc_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    enc_cell = tf.nn.rnn_cell.DropoutWrapper(enc_cell, output_keep_prob=0.5)

    outputs, enc_states = tf.nn.dynamic_rnn(enc_cell, enc_input, dtype=tf.float32)

## 디코더 셀을 구한다
with tf.variable_scope('decode'):
    dec_cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    dec_cell = tf.nn.rnn_cell.DropoutWrapper(dec_cell, output_keep_prob=0.5)

    # Seq2Seq 모델은 인코더 셀의 최종 상태값을 디코더 셀의 초기 상태값으로 넣어주는 것이 핵심!
    outputs, dec_states = tf.nn.dynamic_rnn(dec_cell, dec_input, initial_state=enc_states, dtype=tf.float32)

model = tf.layers.dense(outputs, n_class, activation=None)
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model, labels=targets))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.dense instead.


In [4]:
## 신경망 모델 학습
sess = tf.Session()
sess.run(tf.global_variables_initializer())

input_batch, output_batch, target_batch = make_batch(seq_data)

for epoch in range(total_epoch):
    _, loss = sess.run([optimizer, cost], feed_dict={enc_input:input_batch, dec_input:output_batch, targets:target_batch})
    print('Epoch :', '%04d'%(epoch+1), 'cost =', '{:.6f}'.format(loss))
print('최적화 완료!')

Epoch : 0001 cost = 3.820143
Epoch : 0002 cost = 2.823423
Epoch : 0003 cost = 1.527608
Epoch : 0004 cost = 0.948889
Epoch : 0005 cost = 0.703238
Epoch : 0006 cost = 0.308381
Epoch : 0007 cost = 0.249207
Epoch : 0008 cost = 0.179625
Epoch : 0009 cost = 0.156084
Epoch : 0010 cost = 0.142527
Epoch : 0011 cost = 0.204492
Epoch : 0012 cost = 0.201009
Epoch : 0013 cost = 0.216648
Epoch : 0014 cost = 0.294274
Epoch : 0015 cost = 0.071302
Epoch : 0016 cost = 0.215743
Epoch : 0017 cost = 0.046083
Epoch : 0018 cost = 0.273368
Epoch : 0019 cost = 0.017687
Epoch : 0020 cost = 0.038841
Epoch : 0021 cost = 0.043461
Epoch : 0022 cost = 0.089233
Epoch : 0023 cost = 0.034158
Epoch : 0024 cost = 0.006335
Epoch : 0025 cost = 0.035807
Epoch : 0026 cost = 0.009343
Epoch : 0027 cost = 0.023703
Epoch : 0028 cost = 0.011809
Epoch : 0029 cost = 0.003390
Epoch : 0030 cost = 0.002263
Epoch : 0031 cost = 0.008880
Epoch : 0032 cost = 0.008324
Epoch : 0033 cost = 0.004574
Epoch : 0034 cost = 0.002277
Epoch : 0035 c

In [5]:
## 번역 테스트
# 단어를 입력받아 번역 단어를 예측하고 디코딩하는 함수
def translate(word):
    # 이 모델은 입력값과 출력값 데이터로 [영어단어, 한글단어]를 사용하지만,
    # 예측시에는 한글단어를 알지 못하므로, 디코더의 입출력값을 의미없는 값이 P값으로 채운다
    # ['word', 'PPPP']
    seq_data = [word, 'P'*len(word)]
    input_batch, output_batch, target_batch = make_batch([seq_data])

    # 결과가 [batch_size, time_step, input]으로 나오기 때무에,
    # 2번째 차원인 input 차원을 argmax로 취해 가장 확률이 높은 글자를 예측값으로 만든다
    prediction = tf.argmax(model, 2)

    result = sess.run(prediction, feed_dict={enc_input:input_batch, dec_input:output_batch, targets:target_batch})

    # 결과 값인 숫자의 인덱스에 해당하는 글자를 가져와 글자 배열을 만든다
    decoded = [char_arr[i] for i in result[0]]

    # 출력의 끝을 의미하는 'E' 이후의 글자들을 제거하고 문자열로 만든다
    end = decoded.index('E')
    translated = ''.join(decoded[:end])
    return translated

In [6]:
print('\n === 번역 테스ㅌ === ')

print('word ->', translate('word'))
print('wodr ->', translate('wodr'))


 === 번역 테스ㅌ === 
word -> 단어
wodr -> 나무
